In [ ]:
"""
\********************************************************************************
* Copyright (c) 2024 the Qrisp authors
*
* This program and the accompanying materials are made available under the
* terms of the Eclipse Public License 2.0 which is available at
* http://www.eclipse.org/legal/epl-2.0.
*
* This Source Code may also be made available under the following Secondary
* Licenses when the conditions for such availability set forth in the Eclipse
* Public License, v. 2.0 are satisfied: GNU General Public License, version 2
* with the GNU Classpath Exception which is
* available at https://www.gnu.org/software/classpath/license.html.
*
* SPDX-License-Identifier: EPL-2.0 OR GPL-2.0 WITH Classpath-exception-2.0
********************************************************************************/
"""

from state_preparation_parametrized import *
from qrisp.quantum_backtracking import OHQInt
from classical_cost_func import cost_function, format_coeffs, cost_symp
import numpy as np
import matplotlib.pyplot as plt
from encode_PBS_problem import PBS_graph, N, M, cost_coeff

tot_coeff = format_coeffs(cost_coeff, N)

####################
# Compute optimal solution via unstructured search (brute force method)
####################

qtype = OHQInt(N)
q_array_2 = QuantumArray(qtype = qtype, shape = (M))

uniform_state = prepare_pbs_state(PBS_graph, 0, N, q_array_2)
meas_res = uniform_state.get_measurement()

cl_cost = cost_function(tot_coeff,PBS_graph)

solutions = {}
for k,v in meas_res.items():
    c = cl_cost({k:1})  
    solutions[k] = c
sorted_solutions = sorted(solutions.items(), key=lambda item: item[1])
min_cost = sorted_solutions[0][1]
optimal_solution = sorted_solutions[0][0]

print('###  Best assignment (brute force): ###')
print('   Cost:  ',sorted_solutions[0][1])
print('   State: ',sorted_solutions[0][0])
print('   Number of admissible states: ',len(sorted_solutions))

####################
####################

qtype = OHQInt(N)
q_array = QuantumArray(qtype = qtype, shape = (M))

# Cost function as SymPy polynomial
C,S = cost_symp(tot_coeff,M,N,PBS_graph)
ord_symbs=list(S.values())

def cost_op(q_array, gamma):
    app_sb_phase_polynomial(q_array, C, ord_symbs, t=-gamma)

# Initialization function
init_func = pbs_state_init(PBS_graph, 0 , N)

# Inverse state preparation operator
def inv_init_func(q_array):
    with invert():
        init_func(q_array)

# Define mixer operator
def mixer_op(q_array, beta):
    with conjugate(inv_init_func)(q_array):
        for i in range(len(q_array)):
            mcp(beta, q_array[i], ctrl_state = 0)

# Classical cost function
#values = []
cl_cost = cost_function(tot_coeff,PBS_graph)

# Define QAOA problem
from qrisp.qaoa import *
qaoaPBS = QAOAProblem(cost_operator=cost_op ,mixer=mixer_op, cl_cost_function=cl_cost, init_type='tqa')
qaoaPBS.set_init_function(init_func)
#depth = 3
#res = qaoaPBS.run(q_array, depth,  max_iter = 100)
benchmark = qaoaPBS.benchmark(q_array, [0,1,2,3,4], [5000], [100], optimal_solution, init_type='tqa')

#benchmark.save("data/constrainedQAOA_(1,0),(2,0),(3,1)_4")

In [ ]:
def ProbabilityApproximation10(res,cl_cost,optimal_solution):
    prob = 0
    optiomal_cost = cl_cost({optimal_solution:1})
    for k,v in res.items():
        c = cl_cost({k:1})
        if c < 1.1*optiomal_cost:
            prob += v
    return prob

def Approximation(res,cl_cost,optimal_solution):
    return cl_cost(res)/cl_cost(optimal_solution)

def ProbabilityArrgegated(res,cl_cost):
    costs = list(set([round(cl_cost({item[0]:1.0}),2) for item in res.items()]))
    probs = [0]*len(costs)
    for k,v in res.items():
        cost = round(cl_cost({k:1.0}),2)
        probs[costs.index(cost)] += v
    return costs, probs

cl_cost = benchmark.cost_function
optimal_solution = benchmark.optimal_solution
layers = benchmark.layer_depth
results = benchmark.counts


x1, y1 = ProbabilityArrgegated(results[2],cl_cost)

x2 = layers
y2 = [ProbabilityApproximation10(res,cl_cost,optimal_solution) for res in results]


In [ ]:
import numpy as np 
import scipy
import matplotlib.pyplot as plt 

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12.5, 4))

# First Plot (ProbabilityArrgegated)
ax1.set_xlabel("Cost", fontsize=15, color="#444444", fontname="Segoe UI")
ax1.set_ylabel("Probability", fontsize=15, color="#444444", fontname="Segoe UI")
ax1.set_xticks(ticks = range(8, 27, 4))
ax1.bar(x1, y1, width=0.1, color='#20306f')
#ax1.legend(fontsize=14, labelcolor='linecolor')
ax1.grid()


# Second Plot (ProbabilityApproximation10)
ax2.set_xlabel("Layers", fontsize=15, color="#444444", fontname="Segoe UI")
ax2.set_ylabel("Probability", fontsize=15, color="#444444", fontname="Segoe UI")
ax2.set_xticks(ticks = range(0, 5, 1))
ax2.plot(x2, y2, c='#20306f', marker="o", linestyle='solid', linewidth=3, zorder=3)
#ax2.legend(fontsize=14, labelcolor='linecolor')
ax2.grid()

# Show both plots side by side
plt.tight_layout()

plt.show()